In [346]:
#objective:
#to see if roulette croupier has muscle memory when spinning the ball

#Todo:
#gather data

#process data
#determine if the ball is within expected half of roulette wheel
    #determine what is correct half
        #determine EXPECTED DISTANCE
        #determine EXPECTED RESULT
        #determine CORRECT_HALF? (Boolean)


#list of correct guess

#Other Notes:
    #take into account roulette croupier spin clockwise (CW) and anticlockwise (CW) in alternate.
        #to take reference from previous same direction spin
    #margin of error is half of the wheel
    #to determine if 

In [347]:
#import libraries
import pandas as pd
import numpy as np

In [348]:
#import data
df1 = pd.read_excel('Raw Data\Dataset1.xlsx')
df2 = pd.read_excel('Raw Data\Dataset2.xlsx')
df3 = pd.read_excel('Raw Data\Dataset3.xlsx')
# df1.head(10)
# df2.head(10)
# df3.head(10)

In [349]:
#initializaing variables
Roulette_Wheel = [0,32,15,19,4,21,2,25,17,34,6,27,13,36,11,30,8,23,10,5,24,16,33,1,20,14,31,9,22,18,29,7,28,12,35,3,26]
Roulette_Wheel_Extended2 =  Roulette_Wheel + Roulette_Wheel
Roulette_Wheel_Extended2_Reverse = list(reversed(Roulette_Wheel_Extended2))
Roulette_Wheel_Extended3 = Roulette_Wheel + Roulette_Wheel + Roulette_Wheel
# df_main = pd.DataFrame(columns = ['SN','RESULT','BALL_DIRECTION','DEALER_GENDER','PPR','PR','ED','ER','CORRECT_HALF']) #create main dataframe
df_main = pd.DataFrame()

# Process

#### Functions

In [350]:
#function to drop unnecessary columns
def func_df_drop_col(df):
    df = df.drop(columns=['DEALER_GENDER','SOURCE'])
    return df

# testing
# df = func_df_drop_col(df1)
# df

In [351]:
#function to add PPR (previous previous result) to df
def func_df_add_ppr(df):
    for i in range(4,len(df)):
        df.loc[i,'PPR'] = int(df.loc[i-4,'RESULT'])
    return df

In [352]:
#function to add PPR (previous previous result) to df
def func_df_add_pr(df):
    #add pr
    for i in range(4,len(df)):
        df.loc[i,'PR'] = int(df.loc[i-2,'RESULT'])
    return df

In [353]:
#function to remove unnecessary rows
def func_df_remove_nan_rows(df):
    df = df.loc[4:len(df)].copy() #copy to avoid chain assignment warning
    df = df.reset_index(drop=True) #to reset index after slicing
    return df

In [354]:
#Function to calculate expected distance
def func_expected_distance(ppr,pr,dir):
        #ppr means previous previous result, pr means previous result, dir means direction
        if dir == 'CW':
                ppr_index = Roulette_Wheel_Extended2.index(ppr,0,37)
                # print(ppr_index)
                pr_index = Roulette_Wheel_Extended2.index(pr,ppr_index,74)
                # print(pr_index)
                expected_distance = pr_index - ppr_index 
                if expected_distance <= 18: 
                       return expected_distance
                else:
                        return expected_distance - 37
        elif dir == 'ACW':
                ppr_index = Roulette_Wheel_Extended2_Reverse.index(ppr,0,37)
                # print(ppr_index)
                pr_index = Roulette_Wheel_Extended2_Reverse.index(pr,ppr_index,74)
                # print(pr_index)
                expected_distance = pr_index - ppr_index 
                if expected_distance <= 18: 
                       return expected_distance
                else:
                        return expected_distance - 37

#test func ed 2
# func_expected_distance(19,21,'ACW')

In [355]:
#Function to add expected distance to df
def func_df_add_expected_distance(df):
    for i in range(0,len(df)):
        # print(df.loc[i,'PPR'])
        # print(df.loc[i,'PR'])
        # print(df.loc[i,'BALL_DIRECTION'])
        df.loc[i,'ED'] = func_expected_distance(df.loc[i,'PPR'],df.loc[i,'PR'],df.loc[i,'BALL_DIRECTION'])
    return df

In [356]:
#function to calculate expected result (based on PR and ED)
def func_expected_result(Previous_Result,Expected_Distance,Ball_Direction):
    pr = Previous_Result
    ed = Expected_Distance
    dir = Ball_Direction
    # pr_index = Roulette_Wheel_Extended2.index(pr,0,37)
    #if ball direction is clockwise
    if dir == 'CW':
        #if positive expected distance
        if ed >= 0:
            expected_result_index = Roulette_Wheel_Extended2.index(pr,0,37)+ed
            expected_result = Roulette_Wheel_Extended2[expected_result_index]
            # print(expected_result)
            return expected_result
        
        #if negative expected distance
        else: 
            expected_result_index = Roulette_Wheel_Extended2.index(pr,37,74)+ed
            expected_result = Roulette_Wheel_Extended2[expected_result_index]
            # print(expected_result)
            return expected_result
    #if ball direction is anti-clockwise
    elif dir == 'ACW':
        #if positive expected distance
        if ed >= 0:
            expected_result_index = Roulette_Wheel_Extended2_Reverse.index(pr,0,37)+ed
            expected_result = Roulette_Wheel_Extended2_Reverse[expected_result_index]
            # print(expected_result)
            return expected_result
        #if negative expected distance
        else: 
            expected_result_index = Roulette_Wheel_Extended2_Reverse.index(pr,37,74)+ed
            expected_result = Roulette_Wheel_Extended2_Reverse[expected_result_index]
            # print(expected_result)
            return expected_result
    else:
        return 'Error in BALL_DIRECTION'

#test func_expected_result
# func_expected_result(32,-7,'ACW')


In [357]:
#function calculate expected result from df
def func_df_add_expected_result(df):
    for i in range(0,len(df)):
        # print(df.loc[i,'PPR'])
        # print(df.loc[i,'PR'])
        # print(df.loc[i,'BALL_DIRECTION'])
        df.loc[i,'ER'] = func_expected_distance(df.loc[i,'PPR'],df.loc[i,'PR'],df.loc[i,'BALL_DIRECTION'])
    return df

In [358]:
    #add expected result to dataframe
def func_add_expected_result_to_df(df):
        for i in range(0,len(df)):
            # print(i)
            df.loc[i,'ER'] = func_expected_result(df.loc[i,'PR'],df.loc[i,'ED'],df.loc[i,'BALL_DIRECTION'])
        return df


In [359]:
#function clean up data type
def func_df_clean_datatype(df):
    try:
        df['PPR'] = df['PPR'].astype(int)
        df['PR'] = df['PR'].astype(int)
        df['ED'] = df['ED'].astype(int)
        df['ER'] = df['ER'].astype(int)
    except:
        pass
   
    return df

In [360]:
# function to calculate correct half
def func_add_correct_half_to_df(df):
    for i in range(0,len(df)):
        expected_result_index = Roulette_Wheel_Extended3.index(df.loc[i,'ER'],37,74)
        tmp_correct_half = Roulette_Wheel_Extended3[expected_result_index-9:expected_result_index+9]
        if df.loc[i,'RESULT'] in tmp_correct_half:
            df.loc[i,'CORRECT_HALF'] = True
        else:
            df.loc[i,'CORRECT_HALF'] = False
    return df

#### Process

In [361]:
# drop unnecessary columns (i.e. dealer_gender and source)
df1_processed = func_df_drop_col(df1)
df1_processed

,SN,RESULT,BALL_DIRECTION
0,1,19,CW
1,2,15,ACW
2,3,21,CW
3,4,8,ACW
4,5,21,CW
5,6,22,ACW
6,7,4,CW
7,8,25,ACW
8,9,23,CW
9,10,0,ACW


In [362]:
# add PPR(Previous Previous Result) to df
df1_processed = func_df_add_ppr(df1_processed)
df1_processed.head(10)

,SN,RESULT,BALL_DIRECTION,PPR
0,1,19,CW,NaN
1,2,15,ACW,NaN
2,3,21,CW,NaN
3,4,8,ACW,NaN
4,5,21,CW,19.0
5,6,22,ACW,15.0
6,7,4,CW,21.0
7,8,25,ACW,8.0
8,9,23,CW,21.0
9,10,0,ACW,22.0


In [363]:
# add PR(Previous Result) to df
df1_processed = func_df_add_pr(df1_processed)
df1_processed.head(10)

,SN,RESULT,BALL_DIRECTION,PPR,PR
0,1,19,CW,NaN,NaN
1,2,15,ACW,NaN,NaN
2,3,21,CW,NaN,NaN
3,4,8,ACW,NaN,NaN
4,5,21,CW,19.0,21.0
5,6,22,ACW,15.0,8.0
6,7,4,CW,21.0,21.0
7,8,25,ACW,8.0,22.0
8,9,23,CW,21.0,4.0
9,10,0,ACW,22.0,25.0


In [364]:
# remove nan rows from df
df1_processed = func_df_remove_nan_rows(df1_processed)
df1_processed.head(5)

,SN,RESULT,BALL_DIRECTION,PPR,PR
0,5,21,CW,19.0,21.0
1,6,22,ACW,15.0,8.0
2,7,4,CW,21.0,21.0
3,8,25,ACW,8.0,22.0
4,9,23,CW,21.0,4.0


In [365]:
# add ED(expected distance) to df
df1_processed = func_df_add_expected_distance(df1_processed)
df1_processed.head(5)

,SN,RESULT,BALL_DIRECTION,PPR,PR,ED
0,5,21,CW,19.0,21.0,2.0
1,6,22,ACW,15.0,8.0,-14.0
2,7,4,CW,21.0,21.0,0.0
3,8,25,ACW,8.0,22.0,-12.0
4,9,23,CW,21.0,4.0,-1.0


In [366]:
# clean up datatype df
df1_processed = func_df_clean_datatype(df1_processed)
df1_processed.head(5)

,SN,RESULT,BALL_DIRECTION,PPR,PR,ED
0,5,21,CW,19,21,2
1,6,22,ACW,15,8,-14
2,7,4,CW,21,21,0
3,8,25,ACW,8,22,-12
4,9,23,CW,21,4,-1


In [367]:
# add ER(expected result) to df
df1_processed = func_add_expected_result_to_df(df1_processed)
df1_processed.head(10)

,SN,RESULT,BALL_DIRECTION,PPR,PR,ED,ER
0,5,21,CW,19,21,2,25.0
1,6,22,ACW,15,8,-14,29.0
2,7,4,CW,21,21,0,21.0
3,8,25,ACW,8,22,-12,19.0
4,9,23,CW,21,4,-1,19.0
5,10,0,ACW,22,25,-16,1.0
6,11,24,CW,4,23,13,29.0
7,12,14,ACW,25,0,7,29.0
8,13,33,CW,23,24,3,1.0
9,14,25,ACW,0,14,12,36.0


In [368]:
# clean up datatype df
df1_processed = func_df_clean_datatype(df1_processed)
df1_processed.head(5)

,SN,RESULT,BALL_DIRECTION,PPR,PR,ED,ER
0,5,21,CW,19,21,2,25
1,6,22,ACW,15,8,-14,29
2,7,4,CW,21,21,0,21
3,8,25,ACW,8,22,-12,19
4,9,23,CW,21,4,-1,19


In [369]:
# # df1 - add expected result to df
# df1_processed = func_add_expected_result_to_df(df1_processed)
# df1_processed.head(5)

In [370]:
# add correct half to df
df1_processed = func_add_correct_half_to_df(df1_processed)
df1_processed.head(5)


,SN,RESULT,BALL_DIRECTION,PPR,PR,ED,ER,CORRECT_HALF
0,5,21,CW,19,21,2,25,True
1,6,22,ACW,15,8,-14,29,True
2,7,4,CW,21,21,0,21,True
3,8,25,ACW,8,22,-12,19,True
4,9,23,CW,21,4,-1,19,False


In [371]:
df_main = pd.concat([df_main,df1_processed])
df_main

,SN,RESULT,BALL_DIRECTION,PPR,PR,ED,ER,CORRECT_HALF
0,5,21,CW,19,21,2,25,True
1,6,22,ACW,15,8,-14,29,True
2,7,4,CW,21,21,0,21,True
3,8,25,ACW,8,22,-12,19,True
4,9,23,CW,21,4,-1,19,False
5,10,0,ACW,22,25,-16,1,False
6,11,24,CW,4,23,13,29,False
7,12,14,ACW,25,0,7,29,True
8,13,33,CW,23,24,3,1,True
9,14,25,ACW,0,14,12,36,True


In [372]:
#process dataset 2
df2_processed = func_df_drop_col(df2)
df2_processed = func_df_add_ppr(df2_processed)
df2_processed = func_df_add_pr(df2_processed)
df2_processed = func_df_remove_nan_rows(df2_processed)
df2_processed = func_df_add_expected_distance(df2_processed)
df2_processed = func_df_clean_datatype(df2_processed)
df2_processed = func_add_expected_result_to_df(df2_processed)
df2_processed = func_df_clean_datatype(df2_processed)
df2_processed = func_add_correct_half_to_df(df2_processed)
df_main = pd.concat([df_main,df2_processed])


In [373]:
df_main

,SN,RESULT,BALL_DIRECTION,PPR,PR,ED,ER,CORRECT_HALF
0,5,21,CW,19,21,2,25,True
1,6,22,ACW,15,8,-14,29,True
2,7,4,CW,21,21,0,21,True
3,8,25,ACW,8,22,-12,19,True
4,9,23,CW,21,4,-1,19,False
5,10,0,ACW,22,25,-16,1,False
6,11,24,CW,4,23,13,29,False
7,12,14,ACW,25,0,7,29,True
8,13,33,CW,23,24,3,1,True
9,14,25,ACW,0,14,12,36,True


In [374]:
#process dataset 3
df3_processed = func_df_drop_col(df3)
df3_processed = func_df_add_ppr(df3_processed)
df3_processed = func_df_add_pr(df3_processed)
df3_processed = func_df_remove_nan_rows(df3_processed)
df3_processed = func_df_add_expected_distance(df3_processed)
df3_processed = func_df_clean_datatype(df3_processed)
df3_processed = func_add_expected_result_to_df(df3_processed)
df3_processed = func_df_clean_datatype(df3_processed)
df3_processed = func_add_correct_half_to_df(df3_processed)
df_main = pd.concat([df_main,df3_processed])

In [375]:
df_main

,SN,RESULT,BALL_DIRECTION,PPR,PR,ED,ER,CORRECT_HALF
0,5,21,CW,19,21,2,25,True
1,6,22,ACW,15,8,-14,29,True
2,7,4,CW,21,21,0,21,True
3,8,25,ACW,8,22,-12,19,True
4,9,23,CW,21,4,-1,19,False
5,10,0,ACW,22,25,-16,1,False
6,11,24,CW,4,23,13,29,False
7,12,14,ACW,25,0,7,29,True
8,13,33,CW,23,24,3,1,True
9,14,25,ACW,0,14,12,36,True


In [376]:
#Export for analyzation and visualization in Tableau
df_main.to_excel("output.xlsx") 